In [ ]:
# This script gets all the unique album IDs
import os
import json

folder_path = 'C:\\Users\\Music\\team_project\\team37\\prepopulationStuff\\PythonNotebooksForPrepopulation\\henry'
unique_ids = set() 

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):  
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            for item in data:
                unique_ids.add(item['id']) 

output_file_path = os.path.join(folder_path, 'unique_ids.txt')
with open(output_file_path, 'w') as output_file:
    for id in unique_ids:
        output_file.write(id + '\n')

print(f"Unique IDs have been written to {output_file_path}")

In [ ]:
# This script merges the album IDs from the smaller lists, removing duplicates
def load_list_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            # Reads each line into a list, stripping newline characters
            return [line.strip() for line in file]
    except FileNotFoundError:
        return []

file_path1 = 'dave_unique_ids.txt'
file_path2 = 'henry_unique_ids.txt'
list1 = load_list_from_file(file_path1)
list2 = load_list_from_file(file_path2)
merged_list = list(set(list1 + list2))
output_file_path = 'merged_artist_ids_list.txt'
with open(output_file_path, 'w') as output_file:
    for id in merged_list:
        output_file.write(id + '\n') 

print(merged_list)

In [ ]:
# This script calls the Spotify API to get the album details for each album ID in batches of 20
import requests
import json
import os
import time

def load_album_ids(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print("File not found.")
        return []

def get_albums_info(album_ids, access_token, market='US'):
    url = 'https://api.spotify.com/v1/albums'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'ids': ','.join(album_ids), 'market': market}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve albums information. Status code: {response.status_code}, Response: {response.text}")
        return None

# Your Spotify access token and file paths
access_token = 'BQCsOwImZwg-Y5Axd_x65Uj2rNjHxEgynXP13w7Wm-SLHhmwTQI-4mO6cMPMqkZn1iQXqBseOCcJFk1Tj8n3BG4XfXRLYSSqnbx6eq54_tCmgNXd1TBR3NhSi1-aEbAvzCDj4ac-jaHLdmPc40AaHUKHZFH5eo319zYTH8GvlAIDajbKfv3jfyKGL8tKi1mrUNhfnrK6SMLOp1-praYQsXEUDbiGD7C47SmYkKlBfffPekYHHr8UNcT2SrY7lYYk0BrKw_npxIBUYbXV1SaZWvA7dln6e8mLG5nu5BVl'  # Replace with your actual Spotify access token
album_ids_file_path = 'merged_artist_ids_list.txt'  # The path to your file containing album IDs
output_directory = 'album_information'
os.makedirs(output_directory, exist_ok=True)

# Load the album IDs
album_ids = load_album_ids(album_ids_file_path)

# Process the list in chunks of 20 IDs at a time
for i in range(0, len(album_ids), 20):
    chunk = album_ids[i:i+20]
    albums_info = get_albums_info(chunk, access_token)
    if albums_info:
        # Consider timestamp or some unique attribute for filename uniqueness if required
        timestamp = int(time.time())
        with open(f'{output_directory}/BATCH2albums_info_{i}-{i+len(chunk)}_{timestamp}.json', 'w') as output_file:
            json.dump(albums_info, output_file)

print("Albums information has been saved.")


In [ ]:
# Step 1: Read the lists from the text files and convert them into sets

with open('FULL_ALBUM_LIST.txt', 'r') as file:
    list1 = file.read().strip().split(", ")
    # Converting list items to a set after removing quotes and brackets
    set1 = set(item.strip('[]"') for item in list1)

with open('TO_DO_ALBUM_LIST.txt', 'r') as file:
    list2 = file.read().strip().split("\n")
    # Converting list items to a set after removing quotes and brackets
    set2 = set(item.strip('[]"') for item in list2)

# Step 2: Use set difference to remove IDs from set2 that are present in set1

resulting_set = set2.difference(set1)

# Step 3: Write the resulting set to a new file

# Convert each ID in the resulting set to a string formatted with quotes, then join these with ", "
resulting_list_str = "[" + ", ".join(f'"{id}"' for id in resulting_set) + "]"

# Write the formatted string to a new file
with open('resulting_list_formatted.txt', 'w') as file:
    file.write(resulting_list_str)

print("Completed. The formatted resulting list has been saved to 'resulting_list_formatted.txt'.")